# Agent-based Modeling

Agent-based Modeling (ABM) is a simulation method where the autonomous agents interacting with the environment (space) and/or each other by a set of rules.


The most obvious example of ABM is non-player characters (NPCs) in computer games.


ABM is able to model heterogeneously, i.e. it does not require the environment to be well stirred (as opposed to ODEs), continuous (as opposed to to PDEs), nor need the characteristics of each kind of agents to be identical (as opposed to SSAs). 

This makes ABM more flexible to model individual behaviors. (e.g. traffic jam, disease spread, molecular interactions)

## Elements of ABM

To use `Agents.jl`, we need to define:

- The [**space**](https://juliadynamics.github.io/Agents.jl/stable/api/#Available-spaces) where the agents live
- The [**agents**](https://juliadynamics.github.io/Agents.jl/stable/api/#@agent-macro) with self-defined properties.
- The **model** to hold the `space`, the `agent`s, and other parameters (called `properties`)
- The stepping function `step!()` to tell how the model evolve.

⚠️ *WARNING*

As of 2021-05-26 `abm_plot()` and `abm_video()` [did not work](https://discourse.julialang.org/t/error-no-backend-available-glmakie-cairomakie-wglmakie-when-calling-abm-video-in-interactivedynamics-jl/61626). There might be glitches between `Agents.jl` and `Makie.jl`.

## Could I do ABM from scratch?

## Resources

- [Documentation](https://juliadynamics.github.io/Agents.jl/stable/) of `Agents.jl`.
- [sir-julia](https://github.com/epirecipes/sir-julia) : Various implementations of the classical SIR model in Julia.

In [1]:
using Agents
using Plots
using Random
using DataFrames

# Example 1 : Schelling's segregation model

Taken from `Agents.jl` [tutorial](https://juliadynamics.github.io/Agents.jl/stable/examples/schelling/).

- Space  : 2D grid space with a Chebyshev metric. This leads to 8 neighboring positions per position (except at the edges of the grid).
- Agents : They belong to one of two groups (0 or 1).
- Model : Each position of the grid can be occupied by at most one agent.
- For each step
  - If an agent has at least 3 neighbors belonging to the same group, then it is happy.
  - If an agent is unhappy, it keeps moving to new locations until it is happy.

To define an agent type, we should make a mutable struct derived from `AbstractAgent` with 2 mandatory fields:
- `id::Int` . The identifier number of the agent.
- `pos` . For agents on a 2D grid, the position field should be a tuple of 2 integers.

On top of that, we could define other properties for the agents.

Alternatively, we can use [`@agent`](https://juliadynamics.github.io/Agents.jl/stable/api/#@agent-macro) macro to let `Agents.jl` set up the mandatory fields for us.

```julia
@agent SchellingAgent GridAgent{2} begin
    mood::Bool
    group::Int
end
```

In [2]:
mutable struct SchellingAgent <: AbstractAgent
    id::Int             # The identifier number of the agent
    pos::NTuple{2, Int} # The x, y location of the agent on a 2D grid
    mood::Bool          # whether the agent is happy in its position. (true = happy)
    group::Int          # The group of the agent, determines mood as it interacts with neighbors
end

It is recommeded to write a factory function to make model objects so that it will be easy to recreate and change the parameters.

In [3]:
function make_schelling(; numagents = 320, 
                          griddims = (20, 20), 
                          min_to_be_happy = 3, 
                          seed = 125)
    space = GridSpace(griddims, periodic = false)
    properties = (min_to_be_happy = 3, )
	
	# Random seed for demos to be reproducible
	# Optional in production code
    rng = Random.MersenneTwister(seed)
	
    model = ABM(
        SchellingAgent, space;
        properties, rng, scheduler = Schedulers.randomly
    )

    # populate the model with agents, adding equal amount of the two types of agents
    # at random positions in the model
    for n in 1:numagents
        agent = SchellingAgent(n, (1, 1), false, n < numagents / 2 ? 1 : 2)
        add_agent_single!(agent, model)
    end
    return model
end

make_schelling (generic function with 1 method)

And we define a stepping function in the format

`agent_step!(agent, model)`

In [4]:
function agent_step!(agent::SchellingAgent, model)
    minhappy = model.min_to_be_happy
    count_neighbors_same_group = 0
    # For each neighbor, get group and compare to current agent's group
    # and increment count_neighbors_same_group as appropriately.
    # Here `nearby_agents` (with default arguments) will provide an iterator
    # over the nearby agents one grid point away, which are at most 8.
    for neighbor in nearby_agents(agent, model)
        if agent.group == neighbor.group
            count_neighbors_same_group += 1
        end
    end
    # After counting the neighbors, decide whether or not to move the agent.
    # If count_neighbors_same_group is at least the min_to_be_happy, set the
    # mood to true. Otherwise, move the agent to a random position.
    if count_neighbors_same_group ≥ minhappy
        agent.mood = true
    else
        move_agent_single!(agent, model)
    end
    return
end

agent_step! (generic function with 1 method)

In [5]:
groupcolor(a) = a.group == 1 ? :blue : :orange
groupmarker(a) = a.group == 1 ? :circle : :rect

groupmarker (generic function with 1 method)

In [6]:
model = make_schelling(griddims = (50, 50), numagents = 1800)

anim = @animate for i in 1:25
    step!(model, agent_step!)
    plotabm(model; warn = false, ac = groupcolor, am = groupmarker, as = 5, 
        aspect_ratio=:equal, xlims=(0.0, 50.5), ylims = (0.0, 50.5), 
        title = "Schelling Step $i", size=(600, 600))
end

┌ Warning: Plots.jl recipes have been superseded by InteractiveDynamics.jl
└ @ Agents /home/runner/.julia/packages/Agents/HGute/src/visualization/plot-recipes.jl:61
┌ Warning: Plots.jl recipes have been superseded by InteractiveDynamics.jl
└ @ Agents /home/runner/.julia/packages/Agents/HGute/src/visualization/plot-recipes.jl:61
┌ Warning: Plots.jl recipes have been superseded by InteractiveDynamics.jl
└ @ Agents /home/runner/.julia/packages/Agents/HGute/src/visualization/plot-recipes.jl:61
┌ Warning: Plots.jl recipes have been superseded by InteractiveDynamics.jl
└ @ Agents /home/runner/.julia/packages/Agents/HGute/src/visualization/plot-recipes.jl:61
┌ Warning: Plots.jl recipes have been superseded by InteractiveDynamics.jl
└ @ Agents /home/runner/.julia/packages/Agents/HGute/src/visualization/plot-recipes.jl:61
┌ Warning: Plots.jl recipes have been superseded by InteractiveDynamics.jl
└ @ Agents /home/runner/.julia/packages/Agents/HGute/src/visualization/plot-recipes.jl:61
┌ Warning:

Animation("/tmp/jl_E6AsGC", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000016.png", "000017.png", "000018.png", "000019.png", "000020.png", "000021.png", "000022.png", "000023.png", "000024.png", "000025.png"])

In [7]:
mp4(anim, fps = 5)

┌ Info: Saved animation to 
│   fn = /home/runner/work/mmsb-bebi-5009/mmsb-bebi-5009/docs/abm/tmp.mp4
└ @ Plots /home/runner/.julia/packages/Plots/5kcBO/src/animation.jl:114


Plots.AnimatedGif("/home/runner/work/mmsb-bebi-5009/mmsb-bebi-5009/docs/abm/tmp.mp4")